Steps to get observability

*   Determine metrics (CPU usage, Bytes Transmitted, Bytes Received and Memory usage)
*   Duration of each propagation test (memory, geographical and  



Get all the required libraries

In [2]:
import pandas as pd
import numpy as np
import json, re
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Go to root directory

In [3]:
%cd /content/drive/My Drive/Colab Notebooks/bcgossip/sim/gpbc/cnsim_plosone/memory

/content/drive/My Drive/Colab Notebooks/bcgossip/sim/gpbc/cnsim_plosone/memory


Get memory test duration by loading memory propagation time. This time duration will be used to extract metrics from metrics explorer.  

In [4]:
def get_test_duration(file_path):
    """
    Calculates the duration of a test from a CSV file.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pandas.DataFrame or None: A DataFrame containing the number of nodes,
                                  minimum and maximum datetimes in MYT, or None
                                  if an error occurs.
    """
    # Get source file
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

    try:
        # Remove rows where 'message' ends with "-0" (convergence phase)
        df = df[~df['message'].str.endswith("-0")]

        # Extract number of nodes from 'message'
        df['num_nodes'] = df['message'].str.extract(r'cubaan(\d+)-')[0].astype(int)

        # Group by 'num_nodes' and find the min and max of 'received_timestamp'
        duration_df = df.groupby('num_nodes')['received_timestamp'].agg(['min', 'max']).reset_index()

        # Convert min and max columns to datetime (assuming nanoseconds)
        duration_df['min_datetime_myt'] = pd.to_datetime(duration_df['min'], unit='ns')
        duration_df['max_datetime_myt'] = pd.to_datetime(duration_df['max'], unit='ns')

        # Convert to MYT
        duration_df['min_datetime_myt'] = duration_df['min_datetime_myt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kuala_Lumpur')
        duration_df['max_datetime_myt'] = duration_df['max_datetime_myt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kuala_Lumpur')

        # Format the datetime objects as "M/d/YYYY HH:MM:SS"
        duration_df['min_datetime_myt'] = duration_df['min_datetime_myt'].dt.strftime('%-m/%-d/%Y %H:%M:%S')
        duration_df['max_datetime_myt'] = duration_df['max_datetime_myt'].dt.strftime('%-m/%-d/%Y %H:%M:%S')

        return duration_df

    except KeyError as e:
        print(f"Error: Column not found. {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
## Get default memory test duration
df_def_duration = get_test_duration('test-default-memory-10X.csv')
df_def_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt
0,10,1.742951e+18,1.742951e+18,3/26/2025 08:56:05,3/26/2025 08:57:19
1,50,1.742951e+18,1.742951e+18,3/26/2025 09:00:21,3/26/2025 09:01:42
2,100,1.742951e+18,1.742951e+18,3/26/2025 09:04:32,3/26/2025 09:05:56
3,200,1.742951e+18,1.742951e+18,3/26/2025 09:07:54,3/26/2025 09:09:28
4,400,1.742952e+18,1.742952e+18,3/26/2025 09:11:57,3/26/2025 09:13:52
5,500,1.742952e+18,1.742952e+18,3/26/2025 09:16:24,3/26/2025 09:18:30


In [ ]:
## Get 150Mi memory test duration
df_150Mi_duration = get_test_duration('test-150Mi-10X.csv')
df_150Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46
1,50,1.742910e+18,1.742910e+18,3/25/2025 21:43:24,3/25/2025 21:44:46
2,100,1.742911e+18,1.742911e+18,3/25/2025 21:48:41,3/25/2025 21:50:04
3,200,1.742911e+18,1.742911e+18,3/25/2025 21:52:38,3/25/2025 21:54:10
4,400,1.742911e+18,1.742911e+18,3/25/2025 21:57:05,3/25/2025 21:59:01
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:14:56,3/26/2025 03:16:58


In [ ]:
## Get 300Mi memory test duration
df_300Mi_duration = get_test_duration('test-300Mi-10X.csv')
df_300Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt
0,10,1.742912e+18,1.742912e+18,3/25/2025 22:10:15,3/25/2025 22:11:28
1,50,1.742912e+18,1.742912e+18,3/25/2025 22:15:48,3/25/2025 22:17:09
2,100,1.742912e+18,1.742912e+18,3/25/2025 22:19:23,3/25/2025 22:20:46
3,200,1.742913e+18,1.742913e+18,3/25/2025 22:24:16,3/25/2025 22:25:50
4,400,1.742915e+18,1.742915e+18,3/25/2025 23:02:36,3/25/2025 23:04:33
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:07:38,3/26/2025 03:09:43


In [ ]:
def get_timeseries(file_path):
    """
    Loads a CSV file containing time series data. Converts the 'TimeSeries ID'
    column to datetime objects in Asia/Kuala_Lumpur timezone, and returns the
    processed time series DataFrame.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pandas.DataFrame: The DataFrame with the 'TimeSeries ID' column converted to datetime.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

    # Define the datetime format string to match the data
    datetime_format = '%a %b %d %Y %H:%M:%S GMT%z (Malaysia Time)'

    try:
        # Convert 'TimeSeries ID' to datetime, convert to MYT, and format
        df['TimeSeries ID'] = (pd.to_datetime(df['TimeSeries ID'], format=datetime_format, utc=True)
                            .dt.tz_convert('Asia/Kuala_Lumpur')
                            .dt.strftime('%m/%d/%Y %H:%M:%S'))

        # Convert 'TimeSeries ID' to datetime in df
        df['TimeSeries ID'] = pd.to_datetime(df['TimeSeries ID'])
    except KeyError:
        print("Error: 'TimeSeries ID' column not found in the CSV file.")
        return None
    except ValueError as e:
        print(f"Error: Datetime conversion failed. {e}")
        return None

    return df

a. Observability for Bytes transmitted (with PromQL filtering)
```
sum(rate(kubernetes_io:pod_network_sent_bytes_count{monitored_resource="k8s_pod"}[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.



In [ ]:
# Load Bytes transmitted csv file (from metrix explorer) for default memory, 150Mi and 300Mi tests
# Get these tests min datetime value and max datetime value and key in this in metrics explorer
# together with (a) PromQL command

df_def_transmit = get_timeseries('defaultMemoryBytestransmitted.csv')
# df_def_transmit

df_150Mi_transmit = get_timeseries('150MiMemoryBytestransmitted.csv')
# df_150Mi_transmit

df_300Mi_transmit = get_timeseries('300MiMemoryBytestransmitted.csv')
# df_300Mi_transmit

In [ ]:
# Function to find the max 'telemetry-explorer-0-0' value within a given time range
def find_max_telemetry(min_time, max_time, df_util):
    filtered_df = df_util[(df_util['TimeSeries ID'] >= min_time) & (df_util['TimeSeries ID'] <= max_time)]
    if not filtered_df.empty:
        return filtered_df['telemetry-explorer-0-0'].max()
    else:
        return None  # Return None if no data within the range

In [ ]:
## Getting Bytes transmitted for Default Test

# Apply the function to each row in grouped_df
df_def_duration['bytes_transmit_default'] = df_def_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_def_transmit), axis=1
)

df_def_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_default
0,10,1.742951e+18,1.742951e+18,3/26/2025 08:56:05,3/26/2025 08:57:19,2.228144e+06
1,50,1.742951e+18,1.742951e+18,3/26/2025 09:00:21,3/26/2025 09:01:42,2.596628e+06
2,100,1.742951e+18,1.742951e+18,3/26/2025 09:04:32,3/26/2025 09:05:56,2.614003e+06
3,200,1.742951e+18,1.742951e+18,3/26/2025 09:07:54,3/26/2025 09:09:28,3.241188e+06
4,400,1.742952e+18,1.742952e+18,3/26/2025 09:11:57,3/26/2025 09:13:52,3.955660e+06
5,500,1.742952e+18,1.742952e+18,3/26/2025 09:16:24,3/26/2025 09:18:30,4.643045e+06


In [ ]:
## Getting Bytes transmitted for 150Mi Test

# Apply the function to each row in grouped_df
df_150Mi_duration['bytes_transmit_150Mi'] = df_150Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_150Mi_transmit), axis=1
)

df_150Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_150Mi
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,2.237971e+06
1,50,1.742910e+18,1.742910e+18,3/25/2025 21:43:24,3/25/2025 21:44:46,2.578960e+06
2,100,1.742911e+18,1.742911e+18,3/25/2025 21:48:41,3/25/2025 21:50:04,2.818939e+06
3,200,1.742911e+18,1.742911e+18,3/25/2025 21:52:38,3/25/2025 21:54:10,3.332612e+06
4,400,1.742911e+18,1.742911e+18,3/25/2025 21:57:05,3/25/2025 21:59:01,4.323929e+06
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:14:56,3/26/2025 03:16:58,4.574747e+06


In [ ]:
## Getting Bytes transmitted for 300Mi Test

# Apply the function to each row in grouped_df
df_300Mi_duration['bytes_transmit_300Mi'] = df_300Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_300Mi_transmit), axis=1
)

df_300Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_300Mi
0,10,1.742912e+18,1.742912e+18,3/25/2025 22:10:15,3/25/2025 22:11:28,2.336020e+06
1,50,1.742912e+18,1.742912e+18,3/25/2025 22:15:48,3/25/2025 22:17:09,2.570498e+06
2,100,1.742912e+18,1.742912e+18,3/25/2025 22:19:23,3/25/2025 22:20:46,2.725680e+06
3,200,1.742913e+18,1.742913e+18,3/25/2025 22:24:16,3/25/2025 22:25:50,3.200394e+06
4,400,1.742915e+18,1.742915e+18,3/25/2025 23:02:36,3/25/2025 23:04:33,4.239602e+06
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:07:38,3/26/2025 03:09:43,4.661820e+06


b. Observability for Bytes received (with PromQL filtering)
```
sum(rate(kubernetes_io:pod_network_received_bytes_count{monitored_resource="k8s_pod"}[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.

In [ ]:
# Load Bytes received csv files (from metrix explorer) for default memory, 150Mi and 300Mi tests
# Get these tests min datetime value and max datetime value (from test duration) and key in it to metrics explorer
# together with (b) PromQL command

df_def_received = get_timeseries('defaultMemoryBytesReceived.csv')
# df_def_received

df_150Mi_received = get_timeseries('150MiMemoryBytesReceived.csv')
# df_150Mi_received

df_300Mi_received = get_timeseries('300MiMemoryBytesReceived.csv')
# df_300Mi_received

In [ ]:
## Getting Bytes received for default Test

# Apply the function to each row in grouped_df
df_def_duration['bytes_received_default'] = df_def_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_def_received), axis=1
)
# Select the desired columns using a list
df_def_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_default,bytes_received_default
0,10,1.742951e+18,1.742951e+18,3/26/2025 08:56:05,3/26/2025 08:57:19,2.228144e+06,4.719719e+07
1,50,1.742951e+18,1.742951e+18,3/26/2025 09:00:21,3/26/2025 09:01:42,2.596628e+06,2.091904e+08
2,100,1.742951e+18,1.742951e+18,3/26/2025 09:04:32,3/26/2025 09:05:56,2.614003e+06,7.420401e+06
3,200,1.742951e+18,1.742951e+18,3/26/2025 09:07:54,3/26/2025 09:09:28,3.241188e+06,2.333144e+07
4,400,1.742952e+18,1.742952e+18,3/26/2025 09:11:57,3/26/2025 09:13:52,3.955660e+06,2.219304e+06
5,500,1.742952e+18,1.742952e+18,3/26/2025 09:16:24,3/26/2025 09:18:30,4.643045e+06,2.749042e+06


In [ ]:
## Getting Bytes received for 150Mi test

# Apply the function to each row in grouped_df
df_150Mi_duration['bytes_received_150Mi'] = df_150Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_150Mi_received), axis=1
)

# Remove a column inplace
# df_150Mi_duration.drop('bytes_received_default', axis=1, inplace=True)
# Select the desired columns using a list
df_150Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_150Mi,bytes_received_150Mi
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,2.237971e+06,7.606375e+07
1,50,1.742910e+18,1.742910e+18,3/25/2025 21:43:24,3/25/2025 21:44:46,2.578960e+06,1.046504e+06
2,100,1.742911e+18,1.742911e+18,3/25/2025 21:48:41,3/25/2025 21:50:04,2.818939e+06,1.352381e+06
3,200,1.742911e+18,1.742911e+18,3/25/2025 21:52:38,3/25/2025 21:54:10,3.332612e+06,1.579340e+06
4,400,1.742911e+18,1.742911e+18,3/25/2025 21:57:05,3/25/2025 21:59:01,4.323929e+06,2.120900e+06
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:14:56,3/26/2025 03:16:58,4.574747e+06,2.805925e+06


In [ ]:
## Getting Bytes received for 300Mi Test

# Apply the function to each row in grouped_df
df_300Mi_duration['bytes_received_300Mi'] = df_300Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_300Mi_received), axis=1
)

# Remove a column inplace
# df_150Mi_duration.drop('bytes_received_default', axis=1, inplace=True)
# Select the desired columns using a list
df_300Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_300Mi,bytes_received_300Mi
0,10,1.742912e+18,1.742912e+18,3/25/2025 22:10:15,3/25/2025 22:11:28,2.336020e+06,4.994264e+05
1,50,1.742912e+18,1.742912e+18,3/25/2025 22:15:48,3/25/2025 22:17:09,2.570498e+06,7.363034e+05
2,100,1.742912e+18,1.742912e+18,3/25/2025 22:19:23,3/25/2025 22:20:46,2.725680e+06,1.262570e+06
3,200,1.742913e+18,1.742913e+18,3/25/2025 22:24:16,3/25/2025 22:25:50,3.200394e+06,1.747355e+06
4,400,1.742915e+18,1.742915e+18,3/25/2025 23:02:36,3/25/2025 23:04:33,4.239602e+06,2.437549e+06
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:07:38,3/26/2025 03:09:43,4.661820e+06,3.290057e+06


c. Observability for CPU usage (with PromQL filtering)
```
sum(rate(container_cpu_usage_seconds_total[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.

In [ ]:
# Load CPU usage time series csv files (from metrics explorer) for default memory, 150Mi and 300Mi tests
# Get these tests min datetime value and max datetime value (from test duration) and key in it to metrics explorer
# together with (b) PromQL command

df_def_cpu = get_timeseries('defaultMemoryCPUusage.csv')
# df_def_cpu

df_150Mi_cpu = get_timeseries('150MiMemoryCPUusage.csv')
# df_150Mi_cpu

df_300Mi_cpu = get_timeseries('300MiMemoryCPUusage.csv')
# df_300Mi_cpu

In [ ]:
## Getting CPU usage for default Test

# Apply the function to each row in grouped_df
df_def_duration['cpu_usage_default'] = df_def_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_def_cpu), axis=1
)
# Select the desired columns using a list
df_def_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_default,bytes_received_default,cpu_usage_default
0,10,1.742951e+18,1.742951e+18,3/26/2025 08:56:05,3/26/2025 08:57:19,2.228144e+06,4.719719e+07,9.050088
1,50,1.742951e+18,1.742951e+18,3/26/2025 09:00:21,3/26/2025 09:01:42,2.596628e+06,2.091904e+08,16.925751
2,100,1.742951e+18,1.742951e+18,3/26/2025 09:04:32,3/26/2025 09:05:56,2.614003e+06,7.420401e+06,12.981212
3,200,1.742951e+18,1.742951e+18,3/26/2025 09:07:54,3/26/2025 09:09:28,3.241188e+06,2.333144e+07,25.300223
4,400,1.742952e+18,1.742952e+18,3/26/2025 09:11:57,3/26/2025 09:13:52,3.955660e+06,2.219304e+06,34.152766
5,500,1.742952e+18,1.742952e+18,3/26/2025 09:16:24,3/26/2025 09:18:30,4.643045e+06,2.749042e+06,26.052675


In [ ]:
## Getting CPU usage for 150Mi Test

# Apply the function to each row in grouped_df
df_150Mi_duration['cpu_usage_150Mi'] = df_150Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_150Mi_cpu), axis=1
)
# Select the desired columns using a list
df_150Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_150Mi,bytes_received_150Mi,cpu_usage_150Mi
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,2.237971e+06,7.606375e+07,13.665377
1,50,1.742910e+18,1.742910e+18,3/25/2025 21:43:24,3/25/2025 21:44:46,2.578960e+06,1.046504e+06,11.068263
2,100,1.742911e+18,1.742911e+18,3/25/2025 21:48:41,3/25/2025 21:50:04,2.818939e+06,1.352381e+06,14.862799
3,200,1.742911e+18,1.742911e+18,3/25/2025 21:52:38,3/25/2025 21:54:10,3.332612e+06,1.579340e+06,21.918464
4,400,1.742911e+18,1.742911e+18,3/25/2025 21:57:05,3/25/2025 21:59:01,4.323929e+06,2.120900e+06,16.362784
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:14:56,3/26/2025 03:16:58,4.574747e+06,2.805925e+06,39.677041


In [ ]:
## Getting CPU usage for 300Mi Test

# Apply the function to each row in grouped_df
df_300Mi_duration['cpu_usage_300Mi'] = df_300Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_300Mi_cpu), axis=1
)
# Select the desired columns using a list
df_300Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_300Mi,bytes_received_300Mi,cpu_usage_300Mi
0,10,1.742912e+18,1.742912e+18,3/25/2025 22:10:15,3/25/2025 22:11:28,2.336020e+06,4.994264e+05,8.110648
1,50,1.742912e+18,1.742912e+18,3/25/2025 22:15:48,3/25/2025 22:17:09,2.570498e+06,7.363034e+05,12.435596
2,100,1.742912e+18,1.742912e+18,3/25/2025 22:19:23,3/25/2025 22:20:46,2.725680e+06,1.262570e+06,13.708484
3,200,1.742913e+18,1.742913e+18,3/25/2025 22:24:16,3/25/2025 22:25:50,3.200394e+06,1.747355e+06,16.979048
4,400,1.742915e+18,1.742915e+18,3/25/2025 23:02:36,3/25/2025 23:04:33,4.239602e+06,2.437549e+06,30.300475
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:07:38,3/26/2025 03:09:43,4.661820e+06,3.290057e+06,35.242929


d. Observability for Memory usage (with PromQL filtering)
```
sum(avg_over_time(kubernetes_io:container_memory_used_bytes{monitored_resource="k8s_container"}[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.

In [ ]:
# Load CPU usage time series csv files (from metrics explorer) for default memory, 150Mi and 300Mi tests
# Get these tests min datetime value and max datetime value (from test duration) and key in it to metrics explorer
# together with (b) PromQL command

df_def_memory = get_timeseries('defaultMemoryUsageMemory.csv')
# df_def_memory

df_150Mi_memory = get_timeseries('150MiMemoryUsageMemory.csv')
# df_150Mi_memory

df_300Mi_memory = get_timeseries('300MiMemoryUsageMemory.csv')
# df_300Mi_memory

In [ ]:
## Getting memory usage for default Test

# Apply the function to each row in grouped_df
df_def_duration['memory_usage_default'] = df_def_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_def_memory), axis=1
)
# Select the desired columns using a list
df_def_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_default,bytes_received_default,cpu_usage_default,memory_usage_default
0,10,1.742951e+18,1.742951e+18,3/26/2025 08:56:05,3/26/2025 08:57:19,2.228144e+06,4.719719e+07,9.050088,9329238016
1,50,1.742951e+18,1.742951e+18,3/26/2025 09:00:21,3/26/2025 09:01:42,2.596628e+06,2.091904e+08,16.925751,11595186176
2,100,1.742951e+18,1.742951e+18,3/26/2025 09:04:32,3/26/2025 09:05:56,2.614003e+06,7.420401e+06,12.981212,14180278272
3,200,1.742951e+18,1.742951e+18,3/26/2025 09:07:54,3/26/2025 09:09:28,3.241188e+06,2.333144e+07,25.300223,19956338688
4,400,1.742952e+18,1.742952e+18,3/26/2025 09:11:57,3/26/2025 09:13:52,3.955660e+06,2.219304e+06,34.152766,31008026624
5,500,1.742952e+18,1.742952e+18,3/26/2025 09:16:24,3/26/2025 09:18:30,4.643045e+06,2.749042e+06,26.052675,37403353088


In [ ]:
## Getting memory usage for 150Mi Test

# Apply the function to each row in grouped_df
df_150Mi_duration['memory_usage_150Mi'] = df_150Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_150Mi_memory), axis=1
)
# Select the desired columns using a list
df_150Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_150Mi,bytes_received_150Mi,cpu_usage_150Mi,memory_usage_150Mi
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,2.237971e+06,7.606375e+07,13.665377,9148469248
1,50,1.742910e+18,1.742910e+18,3/25/2025 21:43:24,3/25/2025 21:44:46,2.578960e+06,1.046504e+06,11.068263,11476062208
2,100,1.742911e+18,1.742911e+18,3/25/2025 21:48:41,3/25/2025 21:50:04,2.818939e+06,1.352381e+06,14.862799,15148204032
3,200,1.742911e+18,1.742911e+18,3/25/2025 21:52:38,3/25/2025 21:54:10,3.332612e+06,1.579340e+06,21.918464,20515291136
4,400,1.742911e+18,1.742911e+18,3/25/2025 21:57:05,3/25/2025 21:59:01,4.323929e+06,2.120900e+06,16.362784,31903440896
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:14:56,3/26/2025 03:16:58,4.574747e+06,2.805925e+06,39.677041,37233369088


In [ ]:
## Getting memory usage for 300Mi Test

# Apply the function to each row in grouped_df
df_300Mi_duration['memory_usage_300Mi'] = df_300Mi_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_300Mi_memory), axis=1
)
# Select the desired columns using a list
df_300Mi_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_300Mi,bytes_received_300Mi,cpu_usage_300Mi,memory_usage_300Mi
0,10,1.742912e+18,1.742912e+18,3/25/2025 22:10:15,3/25/2025 22:11:28,2.336020e+06,4.994264e+05,8.110648,9944428544
1,50,1.742912e+18,1.742912e+18,3/25/2025 22:15:48,3/25/2025 22:17:09,2.570498e+06,7.363034e+05,12.435596,13209460736
2,100,1.742912e+18,1.742912e+18,3/25/2025 22:19:23,3/25/2025 22:20:46,2.725680e+06,1.262570e+06,13.708484,15289761792
3,200,1.742913e+18,1.742913e+18,3/25/2025 22:24:16,3/25/2025 22:25:50,3.200394e+06,1.747355e+06,16.979048,20496502784
4,400,1.742915e+18,1.742915e+18,3/25/2025 23:02:36,3/25/2025 23:04:33,4.239602e+06,2.437549e+06,30.300475,32085524480
5,500,1.742930e+18,1.742930e+18,3/26/2025 03:07:38,3/26/2025 03:09:43,4.661820e+06,3.290057e+06,35.242929,37481144320


In [ ]:
# Combine all results to get a table
df_default = df_def_duration[['num_nodes', 'bytes_transmit_default', 'bytes_received_default', 'cpu_usage_default', 'memory_usage_default']]
# df_default

df_150Mi = df_150Mi_duration[['num_nodes', 'bytes_transmit_150Mi', 'bytes_received_150Mi', 'cpu_usage_150Mi', 'memory_usage_150Mi']]
# df_150Mi

df_300Mi = df_300Mi_duration[['num_nodes', 'bytes_transmit_300Mi', 'bytes_received_300Mi', 'cpu_usage_300Mi', 'memory_usage_300Mi']]
# df_300Mi

# Merge DataFrame
df_all = pd.merge(df_default, df_150Mi, on='num_nodes', how='outer')
df_all = pd.merge(df_all, df_300Mi, on='num_nodes', how='outer')
df_all

,num_nodes,bytes_transmit_default,bytes_received_default,cpu_usage_default,memory_usage_default,bytes_transmit_150Mi,bytes_received_150Mi,cpu_usage_150Mi,memory_usage_150Mi,bytes_transmit_300Mi,bytes_received_300Mi,cpu_usage_300Mi,memory_usage_300Mi
0,10,2.228144e+06,4.719719e+07,9.050088,9329238016,2.237971e+06,7.606375e+07,13.665377,9148469248,2.336020e+06,4.994264e+05,8.110648,9944428544
1,50,2.596628e+06,2.091904e+08,16.925751,11595186176,2.578960e+06,1.046504e+06,11.068263,11476062208,2.570498e+06,7.363034e+05,12.435596,13209460736
2,100,2.614003e+06,7.420401e+06,12.981212,14180278272,2.818939e+06,1.352381e+06,14.862799,15148204032,2.725680e+06,1.262570e+06,13.708484,15289761792
3,200,3.241188e+06,2.333144e+07,25.300223,19956338688,3.332612e+06,1.579340e+06,21.918464,20515291136,3.200394e+06,1.747355e+06,16.979048,20496502784
4,400,3.955660e+06,2.219304e+06,34.152766,31008026624,4.323929e+06,2.120900e+06,16.362784,31903440896,4.239602e+06,2.437549e+06,30.300475,32085524480
5,500,4.643045e+06,2.749042e+06,26.052675,37403353088,4.574747e+06,2.805925e+06,39.677041,37233369088,4.661820e+06,3.290057e+06,35.242929,37481144320


In [ ]:
def convert_units(df):
    """
    Converts the numeric columns of a DataFrame to human-readable units.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with converted units.
    """

    def human_readable_bytes(bytes_val):
        if bytes_val is None:
          return "N/A"
        units = ['B', 'KB', 'MB', 'GB', 'TB']
        bytes_val = float(bytes_val)
        i = 0
        while bytes_val >= 1024:
            bytes_val /= 1024
            i += 1
        return f"{bytes_val:.2f} {units[i]}"

    def human_readable_memory(bytes_val):
        if bytes_val is None:
          return "N/A"
        units = ['B', 'KB', 'MB', 'GB', 'TB']
        bytes_val = float(bytes_val)
        i = 0
        while bytes_val >= 1024:
            bytes_val /= 1024
            i += 1
        return f"{bytes_val:.2f} {units[i]}"

    def human_readable_cpu(cpu_val):
        if cpu_val is None:
          return "N/A"
        return f"{cpu_val:.2f}%"

    # Apply conversions
    df['bytes_transmit_default'] = df['bytes_transmit_default'].apply(human_readable_bytes)
    df['bytes_received_default'] = df['bytes_received_default'].apply(human_readable_bytes)
    df['cpu_usage_default'] = df['cpu_usage_default'].apply(human_readable_cpu)
    df['memory_usage_default'] = df['memory_usage_default'].apply(human_readable_memory)
    df['bytes_transmit_150Mi'] = df['bytes_transmit_150Mi'].apply(human_readable_bytes)
    df['bytes_received_150Mi'] = df['bytes_received_150Mi'].apply(human_readable_bytes)
    df['cpu_usage_150Mi'] = df['cpu_usage_150Mi'].apply(human_readable_cpu)
    df['memory_usage_150Mi'] = df['memory_usage_150Mi'].apply(human_readable_memory)
    df['bytes_transmit_300Mi'] = df['bytes_transmit_300Mi'].apply(human_readable_bytes)
    df['bytes_received_300Mi'] = df['bytes_received_300Mi'].apply(human_readable_bytes)
    df['cpu_usage_300Mi'] = df['cpu_usage_300Mi'].apply(human_readable_cpu)
    df['memory_usage_300Mi'] = df['memory_usage_300Mi'].apply(human_readable_memory)

    return df

In [ ]:
# Convert units
df_converted = convert_units(df_all) #Use copy to avoid modifying the original dataframe
df_converted

,num_nodes,bytes_transmit_default,bytes_received_default,cpu_usage_default,memory_usage_default,bytes_transmit_150Mi,bytes_received_150Mi,cpu_usage_150Mi,memory_usage_150Mi,bytes_transmit_300Mi,bytes_received_300Mi,cpu_usage_300Mi,memory_usage_300Mi
0,10,2.12 MB,45.01 MB,9.05%,8.69 GB,2.13 MB,72.54 MB,13.67%,8.52 GB,2.23 MB,487.72 KB,8.11%,9.26 GB
1,50,2.48 MB,199.50 MB,16.93%,10.80 GB,2.46 MB,1021.98 KB,11.07%,10.69 GB,2.45 MB,719.05 KB,12.44%,12.30 GB
2,100,2.49 MB,7.08 MB,12.98%,13.21 GB,2.69 MB,1.29 MB,14.86%,14.11 GB,2.60 MB,1.20 MB,13.71%,14.24 GB
3,200,3.09 MB,22.25 MB,25.30%,18.59 GB,3.18 MB,1.51 MB,21.92%,19.11 GB,3.05 MB,1.67 MB,16.98%,19.09 GB
4,400,3.77 MB,2.12 MB,34.15%,28.88 GB,4.12 MB,2.02 MB,16.36%,29.71 GB,4.04 MB,2.32 MB,30.30%,29.88 GB
5,500,4.43 MB,2.62 MB,26.05%,34.83 GB,4.36 MB,2.68 MB,39.68%,34.68 GB,4.45 MB,3.14 MB,35.24%,34.91 GB
